<div >
    <center> <font size=7 color=red>
    <b>Preventing overfitting </b></font>
    </center>
    <br>
    
</div>

**the techniques for preventing overfitting we will use are :**
*   Regularization
*   Dropout
*   Early Stopping
*   Batch Normalization

In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
# Sweep config for selecting experiment
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'hidden_nodes': {'values': [128]},
        'img_size': {'values': [16]},
        'epochs': {'values': [10]},
        'experiment': {'values': ['dropout_only', 'batchnorm_only', 'full']}
    }
}

sweep_id = wandb.sweep(sweep_config,project="5-flowers-classification")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: u1va0zuq
Sweep URL: https://wandb.ai/ayoubaamir0201-ensam-casablanca/5-flowers-classification/sweeps/u1va0zuq


In [3]:
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        model = keras.Sequential()
        model.add(keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)))

        if config.experiment == 'batchnorm_only':
            model.add(keras.layers.Dense(config.hidden_nodes, use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))

        elif config.experiment == 'dropout_only':
            model.add(keras.layers.Dense(config.hidden_nodes, activation="relu"))
            model.add(keras.layers.Dropout(0.5))

        elif config.experiment == 'full':
            model.add(keras.layers.Dense(config.hidden_nodes,
                                          kernel_regularizer=keras.regularizers.l2(0.01),
                                          use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))
            model.add(keras.layers.Dropout(0.5))

        model.add(keras.layers.Dense(len(CLASS_NAMES), activation="softmax"))

        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]
        if config.experiment == 'full':
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True))

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [4]:
wandb.agent(sweep_id , function = train)

wandb: Agent Starting Run: 9a7qtkju with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: dropout_only
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: ayoubaamir0201 (ayoubaamir0201-ensam-casablanca) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    413/Unknown 155s 370ms/step - accuracy: 0.2469 - loss: 1.6651

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

batch/accuracy,▁▂▄▃▅▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇▇██████
batch/batch_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,████▇▇▇▅▅▅▅▅▄▄▄▄▄▃▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▁▁▁▁▁
epoch/accuracy,▁▃▄▅▆▆▇▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▁▁
epoch/val_accuracy,▁▃▄▅▆▇████
epoch/val_loss,█▄▃▂▁▁▁▁▂▃
batch/accuracy,0.79045


wandb: Agent Starting Run: djyjehdz with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: full
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 94s 226ms/step - accuracy: 0.2504 - loss: 4.1636 - val_accuracy: 0.4378 - val_loss: 3.4170
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 222ms/step - accuracy: 0.3566 - loss: 3.6453 - val_accuracy: 0.4514 - val_loss: 3.2676
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 98s 238ms/step - accuracy: 0.3975 - loss: 3.4151 - val_accuracy: 0.4919 - val_loss: 3.1013
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 344ms/step - accuracy: 0.4317 - loss: 3.1882 - val_accuracy: 0.4865 - val_loss: 2.9735
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 95s 229ms/step - accuracy: 0.4775 - loss: 3.0268 - val_accuracy: 0.5108 - val_loss: 2.8528
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 101s 245ms/step - accuracy: 0.4755 - loss: 2.8740 - val_accuracy: 0.4973 - val_loss: 2.7583
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 93s 226ms/step - accuracy: 0.4871 - loss: 2.7416 - val_accuracy: 0.4919 - val_loss: 2.6667
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 97s 235ms/step - accuracy: 0.5148 - loss

batch/accuracy,▄▁▁▃▃▃▃▃▃▃▇▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████▇█████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▃▄▅▆▆▆▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▂▁▁
epoch/val_accuracy,▁▂▆▅█▆▆█▅▇
epoch/val_loss,█▇▆▅▄▃▃▂▂▁
batch/accuracy,0.55201


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
